In [18]:
import oskar
import matplotlib
#matplotlib.use("Agg")
# pylint: disable=wrong-import-position
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
import numpy

import rascil.processing_components as raproc
import rascil_imager as rascil_imager
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits

ModuleNotFoundError: No module named 'rascil'

In [ ]:
plt.style.use(astropy_mpl_style)

In [ ]:
# Basic settings. (Note that the sky model is set up later.)
params = {
    "simulator": {
        "use_gpus": False
    },
    "observation" : {
        "num_channels": 64,
        "start_frequency_hz": 100e6,
        "frequency_inc_hz": 20e6,
        "phase_centre_ra_deg": 20,
        "phase_centre_dec_deg": -30,
        "num_time_steps": 24,
        "start_time_utc": "01-01-2000 12:00:00.000",
        "length": "12:00:00.000"
    },
    "telescope": {
        "input_directory": "telescope.tm"
    },
    "interferometer": {
        "ms_filename": "visibilities.ms",
        "channel_bandwidth_hz": 1e6,
        "time_average_sec": 10
    }
}
settings = oskar.SettingsTree("oskar_sim_interferometer")
settings.from_dict(params)

# Set the numerical precision to use.
precision = "single"
if precision == "single":
    settings["simulator/double_precision"] = False

# Create a sky model containing three sources from a numpy array.
sky_data = numpy.array([
        [20.0, -30.0, 1, 0, 0, 0, 100.0e6, -0.7, 0.0, 0,   0,   0],
        [20.0, -30.5, 3, 2, 2, 0, 100.0e6, -0.7, 0.0, 600, 50,  45],
        [20.5, -30.5, 3, 0, 0, 2, 100.0e6, -0.7, 0.0, 700, 10, -10]])
sky = oskar.Sky.from_array(sky_data, precision)  # Pass precision here.

# Set the sky model and run the simulation.
sim = oskar.Interferometer(settings=settings)
sim.set_sky_model(sky)
sim.run()

In [ ]:
rascil_imager.main(
    [
        '--ingest_msname','visibilities.ms',
        '--ingest_dd', '0', 
        '--ingest_vis_nchan', '64',
        '--ingest_chan_per_blockvis', '4' ,
        '--ingest_average_blockvis', 'True',
        '--imaging_npixel', '2048', 
        '--imaging_cellsize', '3.878509448876288e-05',
        '--imaging_weighting', 'robust',
        '--imaging_robustness', '-0.5',
        '--clean_nmajor', '0' ,
        '--clean_algorithm', 'mmclean',
        '--clean_scales', '0', '6', '10', '30', '60',
        '--clean_fractional_threshold', '0.3',
        '--clean_threshold', '0.12e-3', '--clean_nmoment' ,'5',
        '--clean_psf_support', '640', '--clean_restored_output', 'integrated'
    ])

In [ ]:
image_file = get_pkg_data_filename('visibilities_nmoment5_cip_deconvolved.fits')
fits.info(image_file)

In [ ]:
image_data = fits.getdata(image_file, ext=0)
numpy_data = numpy.array(image_data)
data = numpy_data[1,0,:,:]
print(data)

plt.figure()
plt.imshow(data, cmap='gray')
plt.colorbar()